## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Victoria,22.2855,114.1577,84.38,87,100,17.00,HK,2022-07-03 14:25:38,overcast clouds
1,1,Avarua,-21.2078,-159.7750,68.05,94,40,1.14,CK,2022-07-03 14:22:58,scattered clouds
2,2,Rikitea,-23.1203,-134.9692,75.31,72,3,20.65,PF,2022-07-03 14:22:50,clear sky
3,3,North Bend,43.4065,-124.2243,56.25,97,100,4.61,US,2022-07-03 14:30:05,overcast clouds
4,4,Ahipara,-35.1667,173.1667,51.22,83,3,5.23,NZ,2022-07-03 14:30:05,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Victoria,22.2855,114.1577,84.38,87,100,17.00,HK,2022-07-03 14:25:38,overcast clouds
2,2,Rikitea,-23.1203,-134.9692,75.31,72,3,20.65,PF,2022-07-03 14:22:50,clear sky
5,5,Puerto Ayora,-0.7393,-90.3518,73.36,92,98,4.00,EC,2022-07-03 14:22:51,overcast clouds
6,6,Vaini,-21.2000,-175.2000,71.76,78,75,11.50,TO,2022-07-03 14:22:51,broken clouds
9,9,Hilo,19.7297,-155.0900,75.27,90,100,3.44,US,2022-07-03 14:25:20,light rain
11,11,Kavieng,-2.5744,150.7967,81.91,79,85,14.25,PG,2022-07-03 14:22:57,overcast clouds
12,12,Katsuura,35.1333,140.3000,76.91,83,100,15.73,JP,2022-07-03 14:30:06,overcast clouds
15,15,Atuona,-9.8000,-139.0333,77.86,77,15,18.81,PF,2022-07-03 14:24:16,few clouds
16,16,Butaritari,3.0707,172.7902,81.63,80,28,16.80,KI,2022-07-03 14:25:20,light rain
18,18,Ponta Do Sol,32.6667,-17.1000,75.00,74,66,1.70,PT,2022-07-03 14:23:01,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        390
City           390
Lat            390
Lng            390
Max Temp       390
Humidity       390
Cloudiness     390
Wind Speed     390
Country        387
Date           390
Description    390
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID        387
City           387
Lat            387
Lng            387
Max Temp       387
Humidity       387
Cloudiness     387
Wind Speed     387
Country        387
Date           387
Description    387
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,
2,Rikitea,PF,75.31,clear sky,-23.1203,-134.9692,
5,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,
6,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,
9,Hilo,US,75.27,light rain,19.7297,-155.0900,
11,Kavieng,PG,81.91,overcast clouds,-2.5744,150.7967,
12,Katsuura,JP,76.91,overcast clouds,35.1333,140.3000,
15,Atuona,PF,77.86,few clouds,-9.8000,-139.0333,
16,Butaritari,KI,81.63,light rain,3.0707,172.7902,
18,Ponta Do Sol,PT,75.00,broken clouds,32.6667,-17.1000,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name
hotel_df.count()

City           387
Country        387
Max Temp       387
Description    387
Lat            387
Lng            387
Hotel Name     387
dtype: int64

In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Central
2,Rikitea,PF,75.31,clear sky,-23.1203,-134.9692,People ThankYou
5,Puerto Ayora,EC,73.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Hilo,US,75.27,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Kavieng,PG,81.91,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
12,Katsuura,JP,76.91,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
15,Atuona,PF,77.86,few clouds,-9.8000,-139.0333,Villa Enata
16,Butaritari,KI,81.63,light rain,3.0707,172.7902,Isles Sunset Lodge
18,Ponta Do Sol,PT,75.00,broken clouds,32.6667,-17.1000,Hotel do Campo


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))